In [1]:
import pandas as pd

import re
import requests
from bs4 import BeautifulSoup
import json

import socket
from urllib3.connection import HTTPConnection


In [2]:
df_train = pd.read_csv("https://lodmedia.hb.bizmrg.com/case_files/768812/train_dataset_train.csv", index_col= 0)
df_test = pd.read_csv("https://lodmedia.hb.bizmrg.com/case_files/768812/test_dataset_test.csv", index_col= 0)
df_all = pd.concat([df_train,df_test], sort=False) #конкатенирую трейн и тест, чтобы одинаково распределить категории и авторов
df_all[7:11]


,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent
document_id,,,,,,,,,,
626cdd489a794743d68e7595sV2ku2yISbSzagjLrginBQ,Губернатор сообщил о попытке самолета ВСУ попа...,2022-04-30 06:57:29,sV2ku2yISbSzagjLrginBQ,[],8.134,5433e5decbb20f277b20eca9,"['547cab2ccbb20f10e6f78407', '543e6d52cbb20fa8...",40870.0,1.038,29.638
620407629a79471871388d31dkFlm-iEQYeqM6cd9yIasQ,Антонов заявил о межпартийном консенсусе США п...,2022-02-09 19:39:13,dkFlm-iEQYeqM6cd9yIasQ,[],0.000,5409f11ce063da9c8b588a12,"['54d510c19a7947184f116809', '58a5cb9c2ae596a9...",9059.0,1.135,32.608
620ccc379a7947f24c5ae12062BHNYaBQxuQFf7YBtqlPw,Опубликовавшая время вторжения The Mirror напи...,2022-02-16 10:11:54,62BHNYaBQxuQFf7YBtqlPw,[],0.000,5409f11ce063da9c8b588a12,"['5883d50f9a7947eac61a7dc5', '5409f17fe063daa0...",4923.0,1.162,54.763
628fbe689a7947e1b08defc7pvilXnBrSLqfn89Qq7bHzA,Медведев обвинил Зеленского в выдвижении «невы...,2022-05-26 18:35:02,pvilXnBrSLqfn89Qq7bHzA,['5a9fa2e39a79470cd086f61a'],4.392,5409f11ce063da9c8b588a12,"['543283d8cbb20f35cbd0f765', '5409f15de063daa0...",13756.0,1.060,31.470


In [3]:
df_all['document'] = df_all.index
df_all.document = df_all.document.apply(lambda x: x[0:24]) #https://www.rbc.ru/rbcfreenews/
df_all.document

document_id
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ    624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw    620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA    620730cf9a7947ab96a44e27
6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w    6262a5889a79470b78c9ca30
626678929a79477ca0101568wuMYES90REuV5YhrN75IXg    626678929a79477ca0101568
                                                            ...           
623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg    623ba6bc9a79470214cb9418
624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ    624fda1f9a79471a9bd102ee
6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww    6257a6a99a79477eea75b86d
624866d59a7947421716ab46WsGDKyGjTJGRh14eR7UVvw    624866d59a7947421716ab46
6236f9129a79477b0ef18ae0ewByZQ1FQBK2dpXyY77rYw    6236f9129a79477b0ef18ae0
Name: document, Length: 10000, dtype: object

In [4]:
df_all.document.value_counts().head(40)
# https://www.rbc.ru/rbcfreenews/621a39ba9a79472784f029d4

621a39ba9a79472784f029d4    31
5e2fe9459a79479d102bada6    18
625568df9a794741e114a762    13
609d11a69a79477cef513697    10
62074b119a7947b0e49b36f7     9
6082cca79a79471b74bf1eda     8
5f8017369a794784407ebca7     6
5afd8e7f9a794772862de199     4
62544e1c9a7947e9895bcb1d     3
624b49b89a794712b4d8a169     2
622e2ada9a79471d2c4de0b6     2
624c19bb9a794752bf834e22     2
621929479a7947d8b8286431     2
61f401bf9a79478e78ad4472     2
628dc8fa9a79471596fd5db7     2
605321259a7947663a3b6ab2     2
620ccb2e9a7947f0fa6e07bd     2
5f92bea39a7947522989725a     2
6144593a9a7947f61043df7f     2
627518769a794781b88bcb04     2
55c3984d9a79470f2c260465     2
5a8eb3149a7947e3e34bba2d     2
609d47119a794795c5f7617d     2
620fd6d99a7947a9ec85178c     2
620230c69a794761414f4016     2
622628279a7947c6c137810d     2
628bb4749a7947f250ed4278     1
620cf6819a79470d7c3fac0e     1
620a4fed9a7947024c0c533e     1
6257e3599a7947980582f9ea     1
6273c2ad9a794703d4ce2c5e     1
628377ca9a79477386659ed2     1
6252f893

In [6]:
HTTPConnection.default_socket_options = (
    HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1),
        (socket.SOL_TCP, socket.TCP_KEEPIDLE, 45), #45
        (socket.SOL_TCP, socket.TCP_KEEPINTVL, 10), #10
        (socket.SOL_TCP, socket.TCP_KEEPCNT, 6) #6
    ]
)

URL = 'https://www.rbc.ru/rbcfreenews/'


def get_soup(url):
    r = requests.get(url).text
    return BeautifulSoup(r, 'lxml')


def save_json(data):
    with open('rbk_data.json', "w") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

texts = []
news_data = {}

for i in df_all.document:
    link = URL + i
    name = link
    news_data[name] = {}
    soup = get_soup(link)

    # Переходим на страницу для дальнейшенго парсинга
    article = soup.find('div', class_='article')
    category = article.find('a', class_='article__header__category') 
#     date = article.find('span', class_='article__header__date').get('content').replace('T', ' ').split('+')[0]
    title = article.find('span', class_='js-slide-title')
    image = article.find('div', class_='article__main-video')
    article_paragraphs = article.find_all('p')
    article_text = ''
    razb = 0
    paragraph_count = 0
    for paragraph in article_paragraphs:
        paragraph_count += 1
        article_text += paragraph.text
        if len(paragraph.text) < 30:
            razb += 1
        if re.search(r'\s\s\s\s\s\s\s', paragraph.text):
            razb += 1
     

    # Заполняем полученными данными news_data
    news_data[name]['link'] = link
#     news_data[name]['date'] = date
    news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', '').replace('\r', '')
    

#     try:
#         news_data[name]['title'] = title.text
#     except AttributeError:
#         news_data[name]['title'] = 'Без заголовка'
    try:
        news_data[name]['category'] = category.text.replace('\n', '')
    except AttributeError:
        news_data[name]['category'] = 'Без категории'
    try:
        news_data[name]['image'] = image.find('video').get('src')
        image = 1
    except AttributeError:
        news_data[name]['image'] = 'Без обложки'
        image = 0
    try: 
        news_data[name]['razb'] = razb
    except AttributeError:
        news_data[name]['razb'] = 0
        
    texts.append([i,news_data[name]['text'],razb])

#     save_json(news_data)

In [7]:
texts

[['624ac09c9a7947db3d80c98e',
  'Совет управляющих Европейского банка реконструкции и развития (ЕБРР) проголосовал за приостановку доступаРоссии и Белоруссии к своим финансам и экспертизе,сообщила пресс-служба финансовой организации.«Доступ Российской Федерации и Белоруссии к ресурсам банка будет официально и немедленно приостановлен после вступления в силу соответствующего решения управляющих»,— указано в сообщении.В банке пояснили, что ограничения также запрещают участие банка в любых проектахРоссии или Белоруссии, в том числе в рамках технического сотрудничества.ЕБРР такжеобъявил, что «оставляет за собой право на приостановку или полное прекращение выборки средств по существующим проектам».После начала спецоперации России на Украине и введения против нее санкций ЕБРР объявил о закрытии своих офисов в Москве и Минске. Банк также согласовал первоначальный пакет помощи в размере €2 млрд для Украины и других стран. В 2017 году ЕБРР анонсировал закрытие всех российских региональных отдел

In [63]:
df_texts = pd.DataFrame(texts, columns=['document', 'text','razb'])


In [64]:
df_texts[df_texts.razb >4]

,document,text,razb
49,620638619a79475745883785,В конце января— начале февраля в связи с распр...,8
76,620bba399a794790a44eb566,"Президент Владимир Путин заявил, что у минских...",5
106,61ff88119a794712318c4e6d,"О том, что сенатор Константин Долгов подготови...",5
109,626a51d99a79475e1c8e30e1,Российские производители колбас и сосисок могу...,5
216,623d7da49a79474c06e6e834,Средние цены на огурцы в российских магазинах ...,5
...,...,...,...
9917,620b6cd19a794762eec523ad,Как стороны оценили отвод части российских вой...,6
9923,624b7dc89a79471fc9076e57,Произошедшее в городе Буча стало операцией вла...,5
9947,6275c7ee9a7947b61420d2ea,Международная реакция на спецоперацию российск...,5
9957,625434929a7947d86e639681,В Брянской области с 11 апреля вводится «желты...,5


In [66]:
df_texts.to_csv('texts_with_razb.csv', index=False)